# Learn Cost Model using Graph Neural Network (GNN) to Predict K-Top Tiles for Tensor Computation Graph

*by Santoshkumar Tongli & Shantanu Satish Bhavsar, December, 12th 2023*

## Introduction

<div style="text-align: justify; text-indent: 4em;"> In this project, we use the TPUGraphs dataset, titled "A Performance Prediction Dataset on Large Tensor Computational Graphs," recently published by Google (August 25, 2023). Our objective is to predict the <b>K-Top tiles for a given computational graph and compilation configuration</b> using this dataset. Specifically, we aim to understand the relationship between the computational graph and the compilation configuration for a provided neural network model. Our goal is to model a cost function that predicts the K-Top tiles, enabling the compilation configuration to achieve optimal performance. </div>

<div style="text-align: justify; text-indent: 4em;"> In our pursuit of understanding this relationship, we initiated the project with a focus on Graph Neural Networks (GNNs). GNNs offer a unique opportunity to contribute to a rapidly evolving field with broad applications, posing open research challenges and the potential to make a significant impact across various industries and domains. The combination of theoretical advances and practical relevance makes GNNs an attractive area for both research and development. </div>

<div style="text-align: justify; text-indent: 4em;"> We started our work based on the foundational program provided by the Google TPUGraphs team. The repository includes three baseline models utilizing TensorFlow-2 and TF-GNN: a Multilayer Perceptron (MLP) model and two GNNs (GraphSAGE and GCN with residual connections). The MLP model embeds all opcodes, concatenates them with node features, sums across all nodes, and further concatenates them with kernel configuration features, feeding into a 3-layer MLP. The combination of graph-level features with node-level information can be accomplished through either late-join or early-join. The former runs the GNN solely on node features, reduces the node embeddings, and concatenates them with the graph (configuration) features. Consequently, multiple configurations over the same graph share the forward and backward pass. The latter replicates the graph features onto every node, grouping configurations per graph, and executing sparse-ops only once per graph (on cube-tensors rather than matrices). The early-join GraphSAGE model closely resembles the original TPU learned cost model [Kaufman, Samuel, 2021].  </div>

<div style="text-align: justify; text-indent: 4em;"> Once playing with the TPUGraph team provided code base, we thought of developing our own codebase to train the model and experiment the things out. We used PyTorch framework to implement our model. We used PyTorch <b>Geometric</b> package to use GNN Convolution layers and other supporting layers. Other then this, out of the box we have also tried learning the data using transforms. We employed transforms, particularly the BertGraphEncoder, to learn the relationships within graphs. These transformers are advanced and incorporate self-attention layers, aiming to capture more general context and prevent overfitting on the training dataset. Further details about the algorithms used and experiments are explained in detail in the methodology section. </div>

## Dataset

<div style="text-align: justify; text-indent: 4em;"> TPUGraphs is a performance prediction dataset on full tensor programs, represented as computational graphs. Each graph represents the main computation of an ML program, which is usually one or many training steps or one inference step. The graphs in the dataset are collected from open-source ML programs, featuring popular models (e.g., ResNet, EfficientNet, Mask R-CNN, and a large variety of Transformer) for a wide range of tasks, e.g., vision, NLP, speech, audio, recommendation, and generative AI. Each data sample contains a computational graph, a compilation configuration, and the execution time when executing the graph when compiled with the given configuration on a Tensor Processing Unit (TPU) v3 [Norman P. Jouppi, 2020], an accelerator for ML workloads. A compilation configuration controls how the XLA compiler [TensorFlow. XLA, 2019] transforms the graph for a specific optimization pass. In particular, the TPUGraphs dataset consists of two collections: (i) <b>Layout</b> and (ii) <b>Tile</b>. Layout configurations control how tensors are laid out in the physical memory, by specifying the dimension order of each input and output of an operation node. A tile configuration controls the tile size of each fused subgraph. </div>

<div style="text-align: justify; text-indent: 4em;"> In this project we focus only on tile configurations. The tile collection contains 13 millions pairs of kernels and configurations, averaging 40 nodes per kernel subgraph. The below table shows the detailed information of the dataset.</div>



<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>The Dataset information</title>
</head>
<body>
    <img src="./images/table.png" alt="Image Description">
</body>
</html>


So, the Tile data is provided in three directories - train, validation, and test. In total, we have been provided with 6988 subgraphs as zipped numpy arrays. In the train set, we have 5709 samples; in the validation set, 676; and in the test set, 844.

Talking about the individual sample of data, it's a zipped numpy array that contains information about the subgraph in the dictionary. It includes the information of the XLA-compilation graph in the graphical representation. In the provided dictionary, the keys are as follows:

<ol>
    <li><b>node_feat</b>: Describes the node’s properties, such as output tensor shape, tensor layout, striding, padding, and operation-specific parameters. All mentioned categorical values are converted to one-hot encoding. The node feature length is 140 and it remains constant across all nodes and graphs.</li>
    <li><b>node_opcode</b>: An integer number specifying a particular operation to be performed, such as convolution, maxpool, etc.</li>
    <li><b>edge_index</b>: Maps the connections/relations between the nodes of the subgraphs. It is provided as a list of connections between two nodes.</li>
    <li><b>config_feat</b>: Contains information about XLA-compiler configuration to run the provided subgraph. It has tile information for individual kernels and other necessary details. For one configuration of the graph, we get a 1x24 vector with integer values. We can run a graph for multiple configurations, so this information will be Nx24.</li>
    <li><b>config_runtime</b>: The time taken to run the inference of the subgraph. It will have runtime values for all configurations, and thus the shape will be Nx1.</li>
    <li><b>config_runtime_normalizers</b>: This is the normalized value of runtime for a given graph across all configurations (Nx1).</li>
</ol>

Using all this information, we need to train a model that can predict the best K-Top Tiles to configure for the graph to run efficiently on the TPU.

## Methods

To train a model capable of predicting the optimal tile for a graph, we have performed experiments using Graph Convolution layers and SAGE Graph (Graph Sample and Aggregation). This section is divided into multiple parts to offer a comprehensive overview of our work.

### Graph Neural Networks - GNN

<div style="text-align: justify; text-indent: 4em;"> Graph Neural Networks (GNNs) are a class of deep learning models designed to operate on graph-structured data. Graphs are mathematical structures composed of nodes and edges, where nodes represent entities, and edges denote relationships between them. GNNs have gained popularity for their ability to capture complex relationships and dependencies within graph data. </div>

<div style="text-align: justify; text-indent: 4em;"> In a typical GNN, each node in the graph is associated with a feature vector, and the model iteratively aggregates information from neighboring nodes to update the node's representation. This process allows GNNs to capture local and global patterns within the graph, making them suitable for various tasks, such as node classification, link prediction, and graph classification. </div>

<div style="text-align: justify; text-indent: 4em;"> To implement this, we utilized PyTorch Geometric, an extension library for PyTorch that facilitates the implementation of Graph Neural Networks (GNNs) and other graph-based models. </div>

### GraphSAGE

<div style="text-align: justify; text-indent: 4em;"> Our methodology is centered around the incorporation of Graph SAGE (Graph Sample and Aggregation), a potent variant of Graph Neural Networks (GNNs) renowned for its effectiveness in learning node representations within graph-structured data. The core of our approach involves representing neural subgraphs, specifically delineating configurations of Tile-XLA hardware, as graphs. In this paradigm, each node encapsulates a distinct aspect of the hardware configuration, containing information about various attributes, such as node features representing hardware characteristics. </div>

<div style="text-align: justify; text-indent: 4em;"> The Graph SAGE algorithm operates through an iterative process of sampling and aggregation, dynamically assimilating information from neighboring nodes. This strategic approach enables our model to discern and propagate intricate dependencies and relationships inherent in the configuration graph. The iterative sampling mechanism ensures that the model captures the nuanced interplay of features, fostering a holistic understanding of the underlying structure. </div>

<div style="text-align: justify; text-indent: 4em;"> Our implementation leverages this graph-based representation to distill meaningful abstractions from the complex interconnections within the neural subgraphs. The ability of Graph SAGE to adaptively aggregate information from diverse local neighborhoods empowers our model to navigate and comprehend the intricate topological patterns present in the Tile-XLA hardware configurations. This nuanced understanding is critical for accurate predictions regarding optimal configurations. Moreover, the graph-based approach provides interpretability, allowing us to trace predictions back to specific nodes and their associated features. The high-dimensional embeddings learned by Graph SAGE encapsulate the essence of the hardware configurations, facilitating efficient downstream processing. Additionally, we employ Multi-Layer Perceptrons (MLPs) to further refine the learned embeddings, enabling the model to capture intricate non-linear relationships and dependencies in the data. </div>

<div style="text-align: justify; text-indent: 4em;"> The choice of hyperparameters, such as the dimensionality of node embeddings, the number of graph convolutional layers, and the architecture of the MLP, has been carefully considered to optimize model performance. Our comprehensive experimentation involved training the model across various configurations, and we meticulously monitored its performance on validation datasets. The inclusion of PyTorch Lightning streamlined the training process, offering scalability and ease of experimentation. </div>

<div style="text-align: justify; text-indent: 4em;"> Post-validation, our trained model was applied to forecast the optimal Top-K tiles for given configuration graphs concerning Tile-XLA on previously unseen test data. The model successfully predicted the K (k = 5) tile values for test samples. Unfortunately, as this is one of the ongoing Kaggle project, for test dataset we dont have ground truth values for comparison and accuracy evaluation.</div>

### Model Architecture

<div style="text-align: justify; text-indent: 4em;"> The foundational architecture of our TileModel is incorporated of Graph Neural Network and Multi-layer perceptron, designed to capture and interpret intricate relationships within Tile-XLA hardware configurations. At its core, TileModel initiates its processing pipeline by utilizing a trainable <b>embedding layer to encapsulate opcodes</b>, a fundamental operation in the landscape of machine learning hardware configurations. This embedding process serves as a pivotal step in transforming discrete opcodes into continuous, trainable representations. Subsequently, the model integrates these opcode embeddings with node features, forming a comprehensive and informative input representation for subsequent layers. The incorporation of opcode embeddings ensures that the model can effectively exploit the nuanced information encoded in these opcodes, facilitating a more profound understanding of the underlying hardware configurations. </div>

<div style="text-align: justify; text-indent: 4em;"> Moreover, the essence of TileModel's architecture lies in the integration of SAGEConv layers, a fundamental component of GraphSAGE (Graph Sample and Aggregation), a variant of Graph Neural Networks (GNNs). These layers are strategically stacked to facilitate graph convolution, enabling the model to discern and exploit intricate dependencies present in the graph-structured data of hardware configurations. Graph convolution plays a pivotal role in this context, as it empowers the model to dynamically aggregate information from neighboring nodes, capturing the complex interplay of features and relationships. The iterative nature of SAGEConv layers ensures that the model navigates the topology of the hardware configuration graphs, progressively refining its understanding through each convolutional step. </div>

<div style="text-align: justify; text-indent: 4em;"> Following the passage of information through the graph convolutional layers, the resultant output is subsequently forwarded to a multi-layer perceptron (MLP). This MLP functions being last part of this model, are responsible for additional processing and generating predictions based on the acquired representations. The incorporation of an MLP, coupled with activation functions, introduces non-linearities to the model, enhancing its capability to capture intricate patterns and dependencies that may exhibit inherent non-linear characteristics within the hardware configurations. The output of the MLP encapsulates the model's prediction concerning optimal configurations for the Tile-XLA hardware. </div>


### Hyperparameters

<div style="text-align: justify; text-indent: 4em;"> The efficacy of our model, TileModel, relies on the optimization of hyperparameters, a critical details need to be assumed and handled (experimented). This involves fine-tuning crucial dimensions and configurations, ensuring that the model adeptly learns relevant and discriminative representations from the complex graph-structured data inherent in Tile-XLA hardware configurations. </div>

<div style="text-align: justify; text-indent: 4em;"> Examining the specifics of the hyperparameters we use in our model, we maximize the opcode embedding size (<span style="color: red;">p_embedding_dim</span>), a crucial parameter that affects the model's ability to represent the intrinsic properties of opcodes. We found that 64 is a good compromise between expressive power and computing efficiency when we set p_embedding_dim in our experiments. Simultaneously, the hidden channel dimensions (<span style="color: red;">p_hidden_channels</span>) govern the depth and complexity of the graph convolutional layers. In our model, we conducted experiments with hidden channels [32, 64, 64, 32], determining that this configuration effectively captured intricate dependencies within the hardware configuration graphs..</div>

<div style="text-align: justify; text-indent: 4em;"> The graph feature size (<span style="color: red;">p_graph_feats</span>) stands as an additional dimension subjected to optimization. In our experiments, configuring <span style="color: red;">p_graph_feats</span> to 128 yielded an optimal point, impacting the breadth of information that SAGEConv layers can capture and process. This dimension plays a crucial role in determining the richness of the learned representations. Concurrently, hidden dimensions (<span style="color: red;">p_hidden_dim</span>) play a pivotal role in molding the internal representations of the model, establishing a trade-off between expressiveness and generalization. In our experiments, configuring <span style="color: red;">p_hidden_dim</span> to 128 resulted in a model achieving an optimal equilibrium, effectively capturing nuanced details while exhibiting strong generalization performance on unseen data. </div>

<div style="text-align: justify; text-indent: 4em;"> The architecture of the multi-layer perceptron (MLP) is important layer and impact the output values based on number of layers and uints we incorporate. The parameter, <span style="color: red;">p_list_dense_layers_uints</span> defines the number of hidden layers we use and uints per layers of the MLP, impacting the model's capacity to capture non-linear patterns and relationships within the learned representations. For chosen configuration of [152, 64, 32, 16, 1] we were able to get the best accuracy. </div>

<div style="text-align: justify; text-indent: 4em;"> During the hyperparameter tuning process, we iteratively explored multiple combinations, utilizing insights acquired from training the model on extensive datasets. The selection of hyperparameter values resulted from experimentation. Notably, dimensions such as <span style="color: red;">p_embedding_dim</span>, <span style="color: red;">p_hidden_channels</span>, <span style="color: red;">p_graph_feats</span>, <span style="color: red;">p_hidden_dim</span>, and <span style="color: red;">p_list_dense_layers_units</span> were selected to minimize the model loss and enhance prediction accuracy. Out of many experiments with hyperparameters, in the table below, we have added 6 different combinations. The table summarizes the multiple experiments we conducted with hyperparameter changes.

<table>
    <tr>
        <td></td>
        <td>Conv Type</td>
        <td>p_embedding_dim</td>
        <td>p_graph_feats</td>
        <td>p_hidden_dim</td>
        <td>p_list_dense_layers_uints</td>
        <td>Accuracy</td>
    </tr>
    <tr>
        <td>Exp 1</td>
        <td>GCNConv</td>
        <td>128</td>
        <td>128</td>
        <td>128</td>
        <td>[152, 64, 32, 16, 1]</td>
        <td>93.93</td>
    </tr>
    <tr>
        <td>Exp 2</td>
        <td>GCNConv</td>
        <td>64</td>
        <td>128</td>
        <td>128</td>
        <td>[152, 64, 32, 16, 1]</td>
        <td>94.4</td>
    </tr>
    <tr>
        <td>Exp 3</td>
        <td>GCNConv</td>
        <td>64</td>
        <td>64</td>
        <td>64</td>
        <td>[88, 64, 32, 16, 1]</td>
        <td>94.38</td>
    </tr>
    <tr>
        <td>Exp 4</td>
        <td>SAGEConv</td>
        <td>128</td>
        <td>128</td>
        <td>128</td>
        <td>[152, 64, 32, 16, 1]</td>
        <td>94.17</td>
    </tr>
    <tr>
        <td>Exp 5</td>
        <td>SAGEConv</td>
        <td>64</td>
        <td>128</td>
        <td>128</td>
        <td>[152, 64, 32, 16, 1]</td>
        <td>94.05</td>
    </tr>
    <tr>
        <td>Exp 6</td>
        <td>SAGEConv</td>
        <td>64</td>
        <td>64</td>
        <td>64</td>
        <td>[88, 64, 32, 16, 1]</td>
        <td>94.26</td>
    </tr>
</table>


### PyTorch Lightning

<div style="text-align: justify; text-indent: 4em;"> PyTorch Lightning is a lightweight PyTorch wrapper that simplifies the process of training and organizing deep learning models. Developed by a team of researchers and engineers, it aims to abstract away the complexities associated with advanced machine learning techniques, allowing researchers to focus more on their models and less on overlapping codes. This project has been developed at NYU CILVR and Facebook AI Research to address challenges in complex AI research. Lightning offical repository has over 320 contributors with a core team of 11 research scientists, PhD students and professional deep learning engineers. It aims to simplify the implementation of advanced techniques like multi-GPU training, 16-bit precision, and TPU training. Designed for scalability, ease of use, and faster iteration in AI research.  In PyTorch Lightning, the core neural network components are encapsulated within <span style="color: red;">pl.LightningModule</span>, replacing <span style="color: red;">nn.Module</span> with LightningModule provides a structured framework for organizing research code. It allows easy transition from standard PyTorch code to a Lightning-based structure. </div>


<div style="text-align: justify; text-indent: 4em;"> To outline the advantages of this module, let's examine the benefits we gain in function modules like data preparation, optimizer, loss, and training modules. In data preparation module, the <span style="color: red;">prepare_data</span> function handles data downloading and preprocessing. It ensures that when using multiple GPUs, data downloading and manipulations are performed only once to avoid duplication. It also abstracts away details to make code cleaner and prevents issues related to multiple GPUs executing the same code. when it comes to optimier Lightning provides the <span style="color: red;">configure_optimizers</span> function to handle optimizer setup. It allows easy selection of optimization algorithms (e.g., Adam) and configuration. It extends to enable support for multiple optimizers, such as those used in GANs. In loss function, there is no such visible change has been seen. The real beauty of using the lighting is in trainer function. In standard PyTorch, users manually write the training loop, which might introduce the potential errors. Lightning abstracts away the boilerplate code associated with training loops, making code cleaner and less error-prone. It's capable of handling complexities like multiple GPUs, gradient clipping, early stopping, checkpointing, TPU training, and 16-bit precision. It mostly abstracts away the details of training, allowing users to focus on high-level model logic. This makes training process organized, making it easier to read, understand, and debug. </div>

<div style="display: flex; justify-content: space-between;">
    <img src="./images/ml_1.png" style="width: 50%;">    
    <img src="./images/ml_2.png" style="width: 50%;">
</div>

The above sample images, shows the difference between using PyTorch and Lightening PyTorch for training loop and Trainer functions.

<div style="text-align: justify; text-indent: 4em;"> We have used PyTorch Lightning to train and evaluate our model. In our implementation, the training process starts by creating an instance of TileModel, which is our neural network designed for Graph Representation Learning. The model incorporates key elements, including trainable embedding layers, SAGEConv layers for graph convolution, and a multi-layer perceptron (MLP) for additional processing. This configuration prepares the model for the training phase. We were able to cast the created model object to PyTorch Lightning by calling LightningWrapper with the model object as a parameter. This allows PyTorch Lightning to handle optimization, collect gradients, and support distributed training if needed. </div>

### Inference and Top Configuration Extraction

<div style="text-align: justify; text-indent: 4em;"> In the inference phase, the trained GNN model is employed on unseen test data, predicting the optimal tile selection. This phase plays a pivotal role in extending the model's learned patterns from the training data to the task of making informed decisions on previously unseen data. In our case, given that it is an ongoing Kaggle project, we dont have labels for the test data. Apparently, the only way to confirm the effectiveness of our model is based on the validation data. </div>

### Contributions
<b>Team Member 1:</b>
<ul>
    <li>Develop and implement the Graph Neural Network (GNN) architecture for the project.</li>
    <li>Integrated model with PyTorch Lightning for training and validation.</li>
    <li>Preprocess and transform the TPUGraphs dataset into a suitable format for GNN training.</li>
    <li>Evaluate and validate the GNN model's predictions, assessing its accuracy and efficiency.</li>
    <li>Investigate and apply state-of-the-art techniques in GNN research to improve model performance.</li>
    <li>Experimented with model by changing Hyper-parameters like, learning rate, optimizer, and with different variants of graph</li>
    <li>Applied the trained model to unseen data for hardware configuration predictions.</li>
</ul>


<b>Team Member 2:</b>
<ul>
    <li>Contribute to the modeling of the cost function, integrating GNN predictions into the final model.</li>
    <li>Optimizing hyperparameters such as Opcode embedding size, Hidden channel dimensions, Graph feature size, Hidden dimensions, Configuration of dense layers in the MLP.</li>
    <li>Evaluation the model results and work on post process tasks - Function to run Inference </li>
    <li>Enabled model compatibility with PyTorch Lightning features like automatic optimization and distributed training.</li>
    <li>Applied trained model to test dataset for inference, to predict top 5 Tile values.</li>
</ul>

## Results

<div style="text-align: justify; text-indent: 4em;"> After training the model using GNNs and conducting multiple experiments, we achieved an accuracy of 94.6%. This accuracy emphasizes the effectiveness of our GNNConv/GraphSAGE-based approach in capturing intricate dependencies within graph-structured data. In our case, the validation accuracy serves as a reliable metric, showcasing the model's proficiency in generalizing to unseen data and making precise predictions. Additionally, in the Model section (1.3.4), we have detailed the experiments conducted with hyperparameters, including relevant tables. Additionally, we have attached some of the training and testing plots generated using TensorBoard. These visualizations provide further insights of the model learning during the training process. </div>

<div style="display: flex; justify-content: space-between;">
    <img src="./images/ml_training_accuracy.png" style="width: 50%;">    
    <img src="./images/ml_val_accuracy.png" style="width: 50%;">
</div>

The above plots shows the train, val accuracy curves. The x-axis indicates <span style="color: red;">Number of steps</span> and the y-axis indicates the <span style="color: red;">Accuracy</span>

<div style="display: flex; justify-content: space-between;">
    <img src="./images/ml_val_loss.png" style="width: 50%;">    
</div>

The above plot shows the validation loss curve. The x-axis indicate <span style="color: red;">Number of steps</span> and the y-axis indicate <span style="color: red;">Loss values</span>

<div style="display: flex; justify-content: space-between;">
    <img src="./images/ml_learning_rate.png" style="width: 50%;">    
</div>

The above plot shows the <span style="color: red;">Decaying Learning Rate Curve</span>. The x-axis indicate <span style="color: red;">Number of steps</span> and the y-axis indicate <span style="color: red;">Learning Rate</span>

<div style="display: flex; justify-content: space-between;">
    <img src="./images/ml_epoch_training.png" style="width: 50%;">    
</div>

The above plot shows the <span style="color: red;">Epochs Curve</span>. The x-axis indicate <span style="color: red;">Number of steps</span> and the y-axis indicate <span style="color: red;">Number of epochs</span>

<div style="display: flex; justify-content: space-between;">
    <img src="./images/ml_train_summary.png" style="width: 50%;">     
    <img src="./images/ml_val_losses.png" style="width: 50%;">     
    
</div>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title></title>
</head>
<body>
    <img src="./images/ml_train_test_accuracy.png" alt="Image Description">
</body>
</html>


The above graphs show the train accuracy, val accuracy and val loss function curves for all 6 experiments we performed with different combinations of hyper parameters.

<div style="text-align: justify; text-indent: 4em;"> As seen from the above experiments, the accuracy obtained ranged from 92% to 95%. Despite conducting numerous experiments on hyperparameters and altering model layers, further improvements in accuracy were not observed. Consequently, we thought of exploring alternate approaches to increase the model accuracy. Given that this project is an ongoing Kaggle project, we turned to the Kaggle website's discussion forum and code section to get some more insights. In this forum, we discovered that some of them were incorporating BERT for model learning. Since the BERT backbone utilizes transformers and they possesses self-attention capabilities, this model may be more capable of capturing relationships between the computational graph and compilation configuration. Referencing code from the Kaggle code section, we started our experiments on the bertencoder model. However, the real challenge was in understanding the complexity of the code, particularly in understanding the transformers part within the BERT encoder. Within a short span of time, we attempted to grasp the fundamental aspects of the program and successfully executed the base code. However, we are still lagging behind in conducting hyperparameter-based experiments and updating the code with evaluation metrics. </div>

<div style="text-align: justify; text-indent: 4em;"> As part of our future work, we intend to utilize a transformer-based model and experiment to enhance accuracy. Additionally, as mentioned in the introduction section, for this project, our focus has been solely on tile-based data. However, incorporating layout-based data and training the model to suggest the optimal graph configuration on hardware, specifically TPU, poses a more complexity and learning this will be more adventures.  </div>

## Conclusions

<div style="text-align: justify; text-indent: 4em;"> In this project, we aimed to learn and understand the relationship between computational graphs and compilation configurations using the TPUGraphs dataset. The primary goal was to predict the K-Top tiles for a given computational graph. We chose Graph Neural Networks (GNNs) as a learning model. We anticipated that implementing GNNs, specifically GraphSAGE, would be important, and optimizing hyperparameters would be a challenging yet essential task. Throughout the project, we implemented and experimented with GNNs, achieving an accuracy of 94.6%. The utilization of GraphSAGE for understanding intricate dependencies within the graph-structured data proved effective. We learned how to handle and preprocess the TPUGraphs dataset, with a focus on the Tile collection. The integration of PyTorch Lightning streamlined our training process, providing scalability and simplifying complex training loops. The project emphasized the significance of hyperparameter tuning, especially in dimensions like opcode embedding size, hidden channel dimensions, and MLP architecture. Transition to BERT Encoder, exploring the use of a transformer-based model, specifically BERT encoder, presented difficulties in understanding the complex code and adapting it to the project's context. Since the project is part of an ongoing Kaggle competition, the lack of ground truth labels for the test dataset posed challenges in evaluating the accuracy of the model. </div>

### References

* [Goodfellow, et al., 2016] Ian Goodfellow and Yoshua Bengio and Aaron Courville, [Deep Learning](http://www.deeplearningbook.org), MIT Press. 2014.
* [Kaufman, Samuel, 2021] Kaufman, Samuel J. and Phothilimthana, Phitchaya Mangpo and Zhou, Yanqi and Mendis, Charith and Roy, Sudip and Sabne, Amit and Burrows, Mike, Proceedings of Machine Learning and Systems, 2021.
* [Norman P. Jouppi, 2020] Norman P. Jouppi, Doe Hyun Yoon, George Kurian, Sheng Li, Nishant Patil, James Laudon, Cliff Young, and David Patterson. A domain-specific supercomputer for training deep neural networks. Commun. ACM, 63(7):67–78, June 2020. ISSN 0001-0782. doi: 10.1145/3360307. URL https://doi.org/10.1145/3360307.
* [TensorFlow. XLA, 2019] TensorFlow. XLA: Optimizing Compiler for TensorFlow. https://www.tensorflow.org/xla. [Online; accessed 19-September-2019]

In [1]:
import io
import nbformat
import glob
nbfile = glob.glob('final_project_report.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, nbformat.NO_CONVERT)
word_count = 0
for cell in nb.cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file final_project_report.ipynb is 4692
